# 参考资料

01: https://cloud.tencent.com/developer/article/1152494 场景文本检测—CTPN算法介绍  
02: https://blog.csdn.net/sinat_33486980/article/details/81099093 faster R-CNN中anchors 的生成过程（generate_anchors源码解析）  
03: https://blog.csdn.net/shenxiaolu1984/article/details/51152614 【目标检测】Faster RCNN算法详解  
04: https://zhuanlan.zhihu.com/p/34757009 <font color="#FF0000">场景文字检测—CTPN原理与实现</font>  
05: https://www.jianshu.com/p/027e9399e699 与CPTN（文字识别网络）作斗争的记录  
06: https://www.zhihu.com/question/265345106/answer/294410307 目标检测中region proposal的作用？  
07: https://blog.csdn.net/u011436429/article/details/80279536 ROI Pooling原理及实现  
08: https://zhuanlan.zhihu.com/p/31426458 <font color="#FF0000">一文读懂Faster RCNN</font>  
09: https://zhuanlan.zhihu.com/p/43534801 <font color="#FF0000">一文读懂CRNN+CTC文字识别</font>  
10: https://blog.csdn.net/zchang81/article/details/78873347 CTPN - 自然场景文本检测

# 参考实现

01: https://github.com/xiaomaxiao/keras_ocr xiaomaxiao/keras_ocr  

# 论文地址

01: https://arxiv.org/abs/1609.03605 Detecting Text in Natural Image with Connectionist Text Proposal Network

# 场景文字检测—CTPN笔记

## CTPN简介

对于复杂场景的文字识别，首先要定位文字的位置，即文字检测。这一直是一个研究热点。  
CTPN是在ECCV 2016提出的一种文字检测算法。CTPN结合CNN与LSTM深度网络，能有效的检测出复杂场景的横向分布的文字，是目前比较好的文字检测算法。

## CTPN网络结构

网络使用keras框架实现

假设输入 N Images：  

1: 首先VGG提取特征，获得大小为 NxHxWxC 的 conv5 feature map  

2: 之后在conv5上做 3×3 的滑动窗口，即每个点都结合周围 3×3 区域特征获得一个长度为 3×3×C 的特征向量。输出 NxHxWx9C 的 feature map，该特征显然只有CNN学习到的空间特征  

3: 再将这个feature map进行Reshape, NxHxWx9C -> (NH)xWx9C  

4: 然后以 Batch=NH 且最大时间长度 T=W 的数据流输入双向LSTM，学习每一行的序列特征。双向LSTM输出 
(NH)xWx256  

5: 再经Reshape恢复形状 (NH)xWx256 -> NxHxWx256 该特征既包含空间特征，也包含了LSTM学习到的序列特征 
  
6: 然后经过“FC”卷积层，变为 NxHxWx256 的特征, “FC”卷积层为核为 1x1 的卷积只会改变 feature map 的厚度  

7: 最后经过类似Faster R-CNN的RPN网络，获得text proposals

In [1]:
import keras
from keras import layers, Input
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras import backend as K
import tensorflow as tf

Using TensorFlow backend.


### 骨干网络模块函数
骨干网络为VGG16，使用VGG16对图片进行特征提取。

In [2]:
def nn_base(input_shape, pretrain_weights_path):
    """
    骨干网络块，使用VGG16底部的卷积层用来对图形进行特征提取
    需要使用预训练模型，请下载vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5
    """
    base_model = VGG16(weights=None, include_top=False, input_shape=input_shape)
    base_model.load_weights(pretrain_weights_path)
    return base_model.input, base_model.get_layer('block5_conv3').output

### reshape特征图到以宽为时间步的形式 NxHxWx9C -> (NH)xWx9C 
构建把feature map进行Reshape, NxHxWx9C -> (NH)xWx9C 的函数

In [3]:
def reshape_to_time_series(input_tensor):
    """
    把输入tensor NxHxWx9C reshape 到 (NH)xWx9C
    """
    tshape = tf.shape(input_tensor)
    output_tensor = tf.reshape(input_tensor, [tshape[0]*tshape[1], tshape[2], tshape[3]])
    return output_tensor

### reshape特征图到高宽通道的形式 (NH)xWx256 -> NxHxWx256
构建Reshape恢复形状 (NH)xWx256 -> NxHxWx256 的函数

In [4]:
def reshape_to_back(input_tensor_list):
    """
    把输入blstm_tensor (NH)xWx256 通过rpn_conv_tensor的形状 reshape 到 NxHxWx256
    """
    blstm, rpn_conv = input_tensor_list
    rshape = tf.shape(rpn_conv)
    output_tensor = tf.reshape(blstm, [rshape[0], rshape[1], rshape[2], -1])
    return output_tensor

### reshape特征图到PRN的形式 NxHxWx20 -> Nx(10HW)x2
构建Reshape到RPN结构 NxHxWx20 -> Nx(10HW)x2 的函数  
NxHxWx20 的意思1是已featrue map每个像素中心坐标生成10个候选框，每个候选框有预测两个类别(前景、背景)  
NxHxWx20 的意思2是已featrue map每个像素中心坐标生成10个候选框，每个候选框有回归2个便宜量(基于每个候选框center_y的偏移量、基于每个候选框h的偏移量)  

In [5]:
def reshape_to_rpn(input_tensor):
    """
    把输入tensor NxHxWx20 reshape 到 Nx(10HW)x2
    """
    tshape = tf.shape(input_tensor)
    output_tensor = tf.reshape(input_tensor, [tshape[0], -1, 2])
    return output_tensor

### 构建完整网络结构

In [6]:
def cptn_model(input_shape, pretrain_weights_path):
    # 输入(None, None, 3) 输出(None/16， None/16, 3) 通过骨干网络进行特征提取 向下取整
    input_tensor, vgg_tensor = nn_base(input_shape, pretrain_weights_path)
    
    # 使得每个点都结合周围(3, 3)的区域特征
    # H*W*C -> H*W*9C
    rpn_conv = layers.Conv2D(
        512*9, (3, 3), padding='same', 
        activation='relu', name='rpn_conv')(vgg_tensor)
    # 此步骤应是对上层网络输出的特征图进行reshape
    # 使特征图的shape变成(NH)*W*C，以(NH)为批次，W为时间步进行学习
    rpn_conv_reshape = layers.Lambda(
        reshape_to_time_series, 
        output_shape=(None, 512*9),
        name='reshape2ts')(rpn_conv)
    # 此步骤是一个双向LSTM，单个反向输出完整的时间步特征图形状为(NH)*W*128
    # 有两个方向会各自得到各自的特征图，按通道堆叠两个特征图 最终得到的特征图形状(NH)*W*256
    blstm = layers.Bidirectional(
        layers.GRU(128, return_sequences=True),
        name='blstm')(rpn_conv_reshape)
    # blstm张量的形状是(NH)*W*256 需要通过rpn_conv的形状来辅助恢复到-> N*H*W*256
    # 以便后面的卷积操作
    blstm_reshape = layers.Lambda(
        reshape_to_back,
        output_shape=(None, None, 256),
        name='reshape2back')([blstm, rpn_conv])
    # 经过“FC”卷积层，变为 N*H*W*512 的特征
    fcnn = layers.Conv2D(
        512, (1, 1), padding='same',
        activation='relu', name='fcnn')(blstm_reshape)
    # 通过卷积提取每个框中是否是前景还是背景的变量
    clas = layers.Conv2D(
        10*2, (1, 1), padding='same',
        activation='linear', name='rpn_class')(fcnn)
    # 将输出类别预测张量形状改变为N*HW10*2
    clas = layers.Lambda(
        reshape_to_rpn, output_shape=(None, 2),
        name='rpn_class_reshape')(clas)
    # 通过卷积提取每个框的的中心y坐标和高度两个值，也是每个框2个变量
    regr = layers.Conv2D(
        10*2, (1, 1), padding='same',
        activation='linear', name='rpn_regress')(fcnn)
    # 将输出框坐标预测张量形状改变为N*HW10*2
    regr = layers.Lambda(
        reshape_to_rpn, output_shape=(None, 2), 
        name='rpn_regress_reshape')(regr)
    
    model = Model(input_tensor, [clas, regr])
    return model

model = cptn_model((None, None, 3), 'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 6 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, None, None, 6 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, None, None, 6 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

## 损失函数

构建RPN的两个损失函数
对于边框偏移量回归使用 smooth L1 loss
对于前背景分类使用 crossentropy loss

In [7]:
def rpn_loss_regr(y_true, y_pred):
    """
    smooth L1 loss
  
    y_ture [1][HXWX10][3] (class1,regr2) 
    y_pred [1][HXWX10][2] (reger2)
    class的值为-1, 0， 1; 
    regr的值为Vc, Vh为预测的anchor和IOU最大的gtbox之间center_y和h的偏移量
    """
    sigma = 9.0
    clas = y_true[0, :, 0]
    regr = y_true[0, :, 1:3]
    # 使用标签的clas类别来选出最合适的anchor的偏移量进行损失函数计算
    regr_keep = tf.where(tf.equal(clas, 1))[:, 0]
    regr_true = tf.gather(regr, regr_keep)
    regr_pred = tf.gather(y_pred[0], regr_keep)
    diff = tf.abs(regr_true - regr_pred)
    less_one = tf.cast(tf.less(diff, 1.0/sigma), 'float32')
    loss = less_one * 0.5 * diff**2 * sigma + tf.abs(1 - less_one) * (diff - 0.5 / sigma)
    # 求出每个可能的候选框的loss
    loss = K.sum(loss, axis=1)
    # 如果没有求出loss那么loss置零，有loss的话算出这个样本的候选框loss的均值
    return K.switch(tf.size(loss) > 0, K.mean(loss), tf.constant(0.0)) 


def rpn_loss_clas(y_true, y_pred):
    """
    softmax loss
    
    y_true [1][HXWX10][1] class 不是one-hot编码
    y_pred [1][HXWX10][2] class 
    """ 
    y_true = y_true[0, :, 0]
    # 选出正负样本，过滤掉无关样本，也就是-1类的框直接忽略了，只留下0类（背景），1类（前景）两类框
    clas_keep = tf.where(tf.not_equal(y_true, -1))[:, 0] 
    clas_true = tf.gather(y_true, clas_keep)
    clas_pred = tf.gather(y_pred[0], clas_keep)
    clas_true = tf.cast(clas_true, 'int64')
    # 求每个标签和对应预测值的交叉熵误差（每个候选框）
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=clas_true, logits=clas_pred) 
    # 如果没有求出loss那么loss置零，有loss的话算出这个样本的候选框前背景loss的均值
    return K.switch(tf.size(loss) > 0, K.clip(K.mean(loss), 0, 10), K.constant(0.0)) 

## RPN标签计算，包含anchor计算

计算对应特征图的基础候选框
1. 因为骨干网络使用的是VGG16，所以骨干网络提取特征后输出的特征图的h,w是原图的16分之1
2. 所以特征图一个像素对应原图16x16的区域，候选框就是以这个区域的中心来生成，候选框的坐标是在原图上的
3. 设定特征图每个像素中心生成10个不同大小的基于原图的候选框
4. 假设特征图大小为(30, 20)那么候选框个数就是30x20x10

### 基础候选框生成器
基础候选框是通过特征图计算出来的，不是预测出来的

In [8]:
def gen_anchor(featuresize, scale):
    # 每个候选点预测10个候选框，这个是候选框基于原图的高度，有10种不同的高度
    heights = [11, 16, 23, 33, 48, 68, 97, 139, 198, 283]
    # 每个候选点预测10个候选框，这个是候选框基于原图的宽度，对于文字预测，宽度相等
    widths  = [16, 16, 16, 16, 16, 16, 16,  16,  16,  16] 

    heights = np.array(heights).reshape(len(heights), 1) # 把数据格式转换为numpy
    widths  = np.array(widths).reshape(len(widths), 1)
    
    # 因为使用的是VGG16的特征图，选取的特征图和原图大小关系刚好是16倍，这里是在选取一个基本anchor
    base_anchor = np.array([0, 0, 15, 15]) 
    # 通过基本anchor求出候选框中心坐标
    cx = (base_anchor[0] + base_anchor[2]) / 2.0 # 宽度一半得到x中心坐标
    cy = (base_anchor[1] + base_anchor[3]) / 2.0 # 高度一半得到y中心坐标

    # 求出在基础候选点的10个候选框坐标
    x1 = cx - widths / 2.0
    y1 = cy - heights / 2.0
    x2 = cx + widths / 2.0
    y2 = cy + heights / 2.0
    base_anchor = np.hstack((x1, y1, x2, y2)) # shape=(10, 4)

    h, w = featuresize
    shift_x = np.arange(0, w) * scale # 基于特征图生成候选点网格
    shift_y = np.arange(0, h) * scale
    
    # 通过基本anchor和基于特征图的候选点网格，生成整个特征图的anchor
    anchor = []
    for i in shift_y:
        for j in shift_x:
            anchor.append(base_anchor + [j, i, j, i])
    return np.array(anchor).reshape((-1, 4)) # shape=(anchor_num, 4)

### 交并比计算相关函数

In [9]:
def cal_iou(box1, box1_area, boxes2, boxes2_area):
    """
    计算交并比 相交的面积和相并的面积的比值

    box1 [x1,y1,x2,y2]            每个anchor
    boxes2 [Msample,x1,y1,x2,y2]  所有的gtbox
    """
    x1 = np.maximum(box1[0], boxes2[:, 0]) # 找出两个框靠左的横坐标的最大的那个 shape=(M,)
    x2 = np.minimum(box1[2], boxes2[:, 2]) # 找出两个框靠右的横坐标的最小的那个 shape=(M,)
    y1 = np.maximum(box1[1], boxes2[:, 1]) # 找出两个框靠上的纵坐标中最大的那个 shape=(M,)
    y2 = np.minimum(box1[3], boxes2[:, 3]) # 找出两个框靠下的纵坐标中最小的那个 shape=(M,)

    intersection = np.maximum(x2 - x1, 0) * np.maximum(y2 - y1, 0) # 如果x2-x1为负数或者y2-y1为负数，那么两个框是不相交的 shape=(M,)
    iou = intersection / (box1_area + boxes2_area - intersection)  # 相交的面积和相并的面积的比值 shape=(M,)
    return iou


def cal_overlaps(boxes1, boxes2):
    """
    计算出每个anchor分别和所有的gtbox的iou

    boxes1 [Nsample,x1,y1,x2,y2]  anchor      shape=(N, 4)
    boxes2 [Msample,x1,y1,x2,y2]  grouth-box  shape=(M, 4)
    """
    area1 = (boxes1[:, 0] - boxes1[:, 2]) * (boxes1[:, 1] - boxes1[:, 3]) # shape=(N,)
    area2 = (boxes2[:, 0] - boxes2[:, 2]) * (boxes2[:, 1] - boxes2[:, 3]) # shape=(M,)

    overlaps = np.zeros((boxes1.shape[0], boxes2.shape[0])) # shape=(N, M)

    for i in range(boxes1.shape[0]):
        overlaps[i][:] = cal_iou(boxes1[i], area1[i], boxes2, area2) # cal_iou返回的shape=(M,) overlaps[i][:]的shape=(M,) 一次计算当前anchor和所有gtbox的iou

    return overlaps # shape=(N, M)

### 把框的(x1, y1, x2, y2)坐标转换为(Vc， Vh)偏移量
通过候选框和真实框计算出候选框需要怎么移动放大才能变成真实框，及反向计算  
计算Vc为y坐标偏移量，Vh为框高度偏移量

In [10]:
def bbox_transfrom(anchors, gtboxes):
    """
    通过候选框和真实框计算出候选框需要怎么移动放大才能变成真实框
    计算Vc为y坐标偏移量，Vh为框高度偏移量
    anchors shape=(N, 4)
    gtboxes shape=(N, 4) 此时的gtboes是有anchor个，是每个anchor对应iou最大的那个gtbox
    """
    cy  = (gtboxes[:, 1] + gtboxes[:, 3]) / 2.0 # shape=(N,)
    cya = (anchors[:, 1] + anchors[:, 3]) / 2.0 # shape=(N,)
    h   = gtboxes[:, 3] - gtboxes[:, 1] + 1.0 # shape=(N,)
    ha  = anchors[:, 3] - anchors[:, 1] + 1.0 # shape=(N,)

    vc = (cy - cya) / ha # shape=(N,)
    vh = np.log(h / ha)  # shape=(N,)
    
    return np.vstack([vc, vh]).transpose() # shape=(2, N) transpose -> shape=(N, 2)


def bbox_transfrom_inv(anchor, regr):
    """
    通过基础anchor和预测出的偏移量，计算出预测出的bbox坐标
    """
    cya = (anchor[:, 1] + anchor[:, 3]) / 2.0
    ha  = (anchor[:, 3] - anchor[:, 1]) + 1

    vc_pred = regr[0, :, 0]
    vh_pred = regr[0, :, 1]

    cy_pred = vc_pred * ha + cya
    h_pred  = np.exp(vh_pred) * ha

    cxa = (anchor[:, 0] + anchor[:, 2]) / 2.0

    x1 = cxa - 16 / 2.0
    y1 = cy_pred - h_pred / 2.0
    x2 = cxa + 16 / 2.0
    y2 = cy_pred + h_pred / 2.0
    bbox = np.vstack([x1, y1, x2, y2]).transpose()

    return bbox

### 计算PRN网络的anchor标签  
这个函数就是通过真实框和基础候选框计算出哪些候选框是前景哪些候选框是背景  
这个就是rpn的核心  

In [11]:
def cal_rpn(imgsize, featuresize, scale, gtboxes):
    """
    计算anchor标签和anchor与gtbox的偏移量
    就是制作RPN网络的标签
    """
    imgh, imgw = imgsize

    base_anchor = gen_anchor(featuresize, scale) # shape=(N, 4)

    overlaps = cal_overlaps(base_anchor, gtboxes) # shape=(N, M)

    labels = np.empty(base_anchor.shape[0]) # shape=(N,)
    labels.fill(-1) # -1为忽略标签

    gt_argmax_overlaps = overlaps.argmax(axis=0) # shape=(M,)

    anchor_argmax_overlaps = overlaps.argmax(axis=1) # shape=(N,)
    # 这里使用了numpy的坐标找值法，按shape的顺序输入坐标组，range(overlaps.shape[0])是选横坐标，anchor_argmax_overlaps选纵坐标
    anchor_max_overlaps = overlaps[range(overlaps.shape[0]), anchor_argmax_overlaps] # shape=(N,)
    
    labels[anchor_max_overlaps > IOU_POSITIVE] = 1 # 给交并比大于阈值的anchor标签设置为1（前景）
    labels[anchor_max_overlaps < IOU_NEGATIVE] = 0 # 给交并比小于与之的anchor标签设置为0（背景）
    labels[gt_argmax_overlaps] = 1 # 各个gtbox交并比最大的anchor标签设置为1（前景），以保证每个gtbox至少有一个对应的anchor

    outside_anchor = np.where(
        (base_anchor[:, 0] < 0) |
        (base_anchor[:, 1] < 0) |
        (base_anchor[:, 2] >= imgw) |
        (base_anchor[:, 3] >= imgh)
        )[0]
    labels[outside_anchor] = -1 # 把超出图片的anchor设置为忽略标签

    # 抑制样本数量
    # 抑制正样本数量
    fg_index = np.where(labels == 1)[0] # 前景标签的下标
    if(len(fg_index) > RPN_POSITIVE_NUM): # 如果选出的前景标签的数量大于设定默认前景标签数量，那么随机选出多余的部分设置值为-1
        labels[np.random.choice(fg_index,len(fg_index) - RPN_POSITIVE_NUM, replace=False)] = -1

    # 抑制负样本数量
    bg_index = np.where(labels==0)[0] # 背景标签的下标
    num_bg = RPN_TOTAL_NUM - np.sum(labels == 1) # 通过总有效候选框数量减去正数量得到应该有的背景数量
    if(len(bg_index) > num_bg):
        labels[np.random.choice(bg_index,len(bg_index) - num_bg, replace=False)] = -1 # 随机选择超出的背景标签设置为-1

    # 把anchor坐标标签转换为Vc和Vh标签
    bbox_targets = bbox_transfrom(base_anchor, gtboxes[anchor_argmax_overlaps, :]) 
    # gtboxes[anchor_argmax_overlaps,:]是找出对应anchor的IOU最大的gtbox

    return [labels, bbox_targets], base_anchor

## 训练数据生成器

### 切分原始的gtbox
因为CTPN的候选框是宽度固定的小框，也不会预测宽度和x坐标  
所以需要先把真实框切分成等宽的小框以便计算rpn标签

In [12]:
def split_gtboxes(gtboxes):
    """
    把标签原始的大框，转换为宽度16像素的小框，以便后面计算anchor
    """
    new_gtboxes = []
    for gtbox in gtboxes:
        x1 = gtbox[0]
        y1 = gtbox[1]
        x2 = gtbox[2]
        y2 = gtbox[3]
        gtbox_w = x2 - x1
        w_steps = math.ceil(gtbox_w / 16)
        for _ in range(w_steps):
            new_gtboxes.append((x1, y1, x1+16, y2))
            x1 += 16
    
    return np.array(new_gtboxes).reshape(-1, 4)

### gtbox读取器

In [13]:
def readtxt(path):
    gtboxes = []
    imgfile = os.path.splitext(os.path.split(path)[-1])[0].split('_')[-1] + '.jpg'
    with open(path, 'r') as f:
        for line in f:
            line = re.sub(r'["\n]', '', line)
            line = re.split(r'\s', line)
            gtboxes.append([int(line[0]), int(line[1]), int(line[2]), int(line[3])])
    return np.asarray(gtboxes), imgfile

### dataloader函数

In [14]:
def gen_sample(textdir, imgdir):
    """
    训练数据生成器
    """
    textfiles = [file for file in os.listdir(textdir) if file.endswith('.txt')]
    random.shuffle(textfiles)
    textfiles = np.array(textfiles)

    i = 0
    end_index = len(textfiles)
    
    while True:
        if i >= end_index:
            i = 0
        textfile = textfiles[i]
        gtbox, imgfile = readtxt(os.path.join(textdir, textfile))
        img = cv2.imread(os.path.join(imgdir, imgfile))
        h, w, _ = img.shape

        # 随机水平翻转
        if np.random.randint(0, 100) > 50:
            img = img[:, ::-1, :]
            newx1 = w - gtbox[:, 2] - 1
            newx2 = w - gtbox[:, 0] - 1
            gtbox[:, 0] = newx1
            gtbox[:, 2] = newx2

        # 把大框切成小框
        gtbox = split_gtboxes(gtbox)

        # 计算出anchor标签
        [clas, regr], _ = cal_rpn((h, w), (int(h/16), int(w/16)), 16, gtbox)

        # 减去imagenet图像平均值，标准化图像
        m_img = img - IMAGE_MEAN
        m_img = m_img[np.newaxis, ...]           # shape=(1, h, w, c)

        regr = np.hstack([clas.reshape(clas.shape[0], 1), regr])

        clas = clas[np.newaxis, ..., np.newaxis] # shape=(1, HW10, 1) [1, HW10, [class]]
        regr = regr[np.newaxis, ...]             # shape=(1, HW10, 3) [1, HW10, [class, Vc, Vh]]

        yield m_img, {'rpn_class_reshape': clas, 'rpn_regress_reshape': regr}

        i += 1

## 文本线构造算法——小框合成大框

预测出来的是许多挨着的小框，需要把这些小框合并为大框
1. 计算每个框的伙伴框(也就是和当前框相邻的框)
2. 构建当前框和伙伴框的关系图表
3. 通过关系图表合并小框到大框

### 查找伙伴框

1. 以当前i框为准，沿着x轴方向向后逐个像素查找k框  
2. 对比每个找到的k框，是否y方向上的交并比在阈值内
3. 如果交并比在阈值内，k框就为i框的伙伴框
4. 返回k框在所有anchor中的index

In [15]:
def meet_v_iou(index1, index2, text_proposals):
    """
    用于检测两个框的垂直方向上的相似和重叠情况
    """
    heights = text_proposals[:, 3] - text_proposals[:, 1] + 1

    def overlaps_vertical(index1, index2):
        """
        求两个框垂直方向上的重叠率
        """
        h1 = heights[index1] # 框1的高
        h2 = heights[index2] # 框2的高
        # 两个框重叠处的y坐标
        y0 = max(text_proposals[index2][1], text_proposals[index1][1]) 
        y1 = min(text_proposals[index2][3], text_proposals[index1][3])
        return max(0, y1 - y0) / min(h1, h2) # 求重叠处的h处以两框中最短的h


    def size_similarity(index1, index2):
        """
        求两个框高度相似度
        """
        h1 = heights[index1]
        h2 = heights[index2]
        return min(h1, h2) / max(h1, h2)

    v_iou = overlaps_vertical(index1, index2) >= MIN_V_OVERLAPS and \
          size_similarity(index1, index2) >= MIN_SIZE_SIM

    return v_iou # bool类型

In [16]:
def get_successions(index, text_proposals, boxes_table, imgsize):
    """
    查找当前的框向后有没有连续的候选框
    找当前框向后的伙伴框
    """
    _, w = imgsize

    box = text_proposals[index] # 获取当前框
    results = []
    # 这里是开始在当前box右边x坐标之后，在gap个像素之内查找有没有候选框
    for right in range(int(box[0]) + 1, min(int(box[0]) + MAX_HORIZONTAL_GAP, w)):
        r_box_indices = boxes_table[right]
        for r_box_index in r_box_indices:
            # 比较右边的框和当前的框垂直方向上的相似度
            # 这函数可以过滤掉x坐标相近但是y坐标相差很大的框，也就是跨行了
            if meet_v_iou(r_box_index, index, text_proposals): 
                results.append(r_box_index)
        if len(results) != 0:
            return results
    return results

### 验证伙伴框

1. 以当前k框为准，沿着x轴方向向前逐个像素查找i框  
2. 对比每个找到的i框，是否y方向上的交并比在阈值内
3. 如果交并比在阈值内，i框就为k框的伙伴框
4. 返回i框在所有anchor中的index
5. 检查这里计算出的i框和之前选定的i框是否是同一个框
6. 如果是同一个框那么可以确定i框和k框一定是伙伴框

In [17]:
def get_successions(index, text_proposals, boxes_table, imgsize):
    """
    查找当前的框向后有没有连续的候选框
    找当前框向后的伙伴框
    """
    _, w = imgsize

    box = text_proposals[index] # 获取当前框
    results = []
    # 这里是开始在当前box右边x坐标之后，在gap个像素之内查找有没有候选框
    for right in range(int(box[0]) + 1, min(int(box[0]) + MAX_HORIZONTAL_GAP, w)):
        r_box_indices = boxes_table[right]
        for r_box_index in r_box_indices:
            # 比较右边的框和当前的框垂直方向上的相似度
            # 这函数可以过滤掉x坐标相近但是y坐标相差很大的框，也就是跨行了
            if meet_v_iou(r_box_index, index, text_proposals): 
                results.append(r_box_index)
        if len(results) != 0:
            return results
    return results

In [18]:
def is_succession_node(index, successions_index, text_proposals, scores, boxes_table):
    """
    这个函数是以之前向后找出的框为基准再向前找符合要求的框
    如果这个框的score比当前框小，那么当前框和向后的框是一个最长连接
    这个应该是以分数为界，切割出一个个连接
    """
    # 以向前查找到的j_index的框向前查找，找到k_index的框
    precursors = get_precursors(successions_index, text_proposals, boxes_table) 
    # 如果scores_index > scores_k_index那么这个序列是最长连接
    if scores[index] >= np.max(scores[precursors]): 
        return True
    return False

### 构建当前框和伙伴框的关系图表

1. 所有小框的左边x坐标为关键字，在x轴上映射一个转换表，这个表可以用像素x坐标查找n个像素内的框
2. 查找伙伴框
3. 通过验证的伙伴框会映射到伙伴关系图表中
4. 多次循环前面2步构建完整的伙伴关系表

In [19]:
def build_graph(text_proposals, scores, imgsize):
    """
    构建框的伙伴关系图表
    """
    _, w = imgsize

    boxes_table = [[] for _ in range(w)]  # 以x轴的顺序建立一个表
    for index, box in enumerate(text_proposals):
        boxes_table[int(box[0])].append(index)  # 以小框左边x坐标，把小框映射到表中
    
    # 构造图表，形状为(N, N)第一维是当前框，第二维是伙伴框，伙伴框因该在当前框的右边
    graph = np.zeros((text_proposals.shape[0], text_proposals.shape[0])) 
    
    for index, box in enumerate(text_proposals):
        successions = get_successions(index, text_proposals, boxes_table, imgsize)
        if len(successions) == 0:
            continue
        # get_successions函数会找一组相邻的框，如果这一组有多个框这里选出的分最高的框
        successions_index = successions[np.argmax(scores[successions])] 
        if is_succession_node(index, successions_index, text_proposals, scores, boxes_table):
            # 确定伙伴关系，当index和successions_index的两个框是伙伴时，图表的这个坐标设置为True
            graph[index, successions_index] = True 
    return graph

### 通过框的伙伴关系图表，合并小框到大框

伙伴关系图表：
1. 假设一共有N个小框
2. 伙伴关系图表M的形状为(N, N) 第一维是当前框的编号，第二维是伙伴框的编号
3. 如果编号i的框有伙伴，那么M\[i, :\]有元素为True， 那为True的位置就i框伙伴的编号
4. 如果编号k的框是其他框的伙伴，那么M\[:, k\]有元素为True，为True的位置编号的框的伙伴就是k框
5. 如果s号框 M\[:, s\] 没有元素为True 且 M\[s, :\] 有元素为True，通过3、4点可以得出s号框不是任何框的伙伴，s号框有伙伴，所以s号框是一个大框的起始
6. 如果e号框 M\[:, e\] 有元素为True 且 M\[e, :\] 没有元素为True，依然通过3、4点可以得出e号框是其他框的伙伴，e号框没有伙伴，所以e号框是一个大框的结尾

In [20]:
def sub_graphs_connected(graph):
    """
    通过框的伙伴关系图表，分出相连的框到一个列表中，这个列表中的小框是属于一个大框的
    """
    sub_graphs = []
    for index in range(graph.shape[0]):
        # not graph[:, index].any() 这个是找的successions_index的框，如果这个框对应的那一列下没有True，那么说明这个坐标之前没有框和它连续
        # graph[index, :].any() 找的是index的框，如果这个框对应的那一行下有True，那么说明这个坐标只有有和它连续的框
        # 因此可以断定这个框是一个起始框
        if not graph[:, index].any() and graph[index, :].any(): # 没有框和index连续，index有连续的框
            v = index # 设置起始框的index
            # 构建子框的列表，列表里面是存放小框的序号，这些小框是属于一个大框的
            sub_graphs.append([v]) 
            while graph[v, :].any(): # v框有伙伴时
                # np.where(graph[v, :])取出了那一行shape=(n,) 
                # 这句代码其实取出的是当前行的横坐标，也就是v对应伙伴的index
                v = np.where(graph[v, :])[0][0]
                # 把伙伴添加到当前子框列表中
                sub_graphs[-1].append(v)
                # 不断的迭代查找v有没有伙伴，有伙伴就继续查找伙伴的伙伴
                # 这样最终找到所有连在一起的框
    return sub_graphs # 返回的是嵌套列表，共两层
                      # 外层是大框，内层是大框由那些小框组成

## 文本线构造算法——计算大框的四个顶点

1. 通过合并框算法，得到大框列表L，L中的元素是小框的编号
2. 拟合L中所有小框的左上点和右下点为一条直线
3. 通过直线方程求解左上右上、左下右下4个点的坐标，得到大框的坐标

In [21]:
def fit_y(X, Y, x1, x2):
    """
    通过X和Y坐标集拟合一条直线
    """
    # 这个句话翻译一下就是X中只有1个坐标，也就是X，Y点集只有一个点，这样就只能返回Y[0]坐标了
    if np.sum(X == X[0]) == len(X): 
        return Y[0], Y[0]

    line = np.poly1d(np.polyfit(X, Y, 1)) # 通过X，Y拟合一个一次方程也就是一条直线
    
    return line(x1), line(x2) # 返回x1,x2时这条直线上y坐标


def threshold(coords, mini, maxi):
    """
    把coords的大小压缩在mini和maxi之间
    """
    return np.maximum(np.minimum(coords, maxi), mini)


def clip_boxes(boxes, im_shape):
    """
    Clip boxes to image boundaries.
    剪切框，使框在图片中
    """
    boxes[:, 0::2] = threshold(boxes[:, 0::2], 0, im_shape[1] - 1)
    boxes[:, 1::2] = threshold(boxes[:, 1::2], 0, im_shape[0] - 1)
    return boxes

## 极大值抑制

对于每个目标，预测出的框大多数情况有多个，极大值抑制就是在这些框中优选出最合适的框作为当前目标的预测框

In [22]:
def nms(dets, thresh):
    """
    极大值抑制，过滤掉和最大值相近的框
    这个算法是以每个框的得分从大到小来进行搜素需要被抑制的框
    """
    x1 = dets[:, 0]
    y1 = dets[:, 1]
    x2 = dets[:, 2]
    y2 = dets[:, 3]
    scores = dets[:, 4]

    areas = (x2 - x1 + 1) * (y2 - y1 + 1) # shape=(N,)
    order = scores.argsort()[::-1] # 从大到小的排序，返回的是index

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i) # 找出得分最大值
        # 求解得分最大值和其他框的相交面积，求出交并比
        xx1 = np.maximum(x1[i], x1[order[1:]]) # shape=(N-1,) 在order[1:]上寻找
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        ovr = inter / (areas[i] + areas[order[1:]] - inter) # shape=(N-1,)
        
        # shape=(N-1)，找出低于阈值的框，因为高于阈值的框和当前得分最高的框重叠度很高
        # 留下交并比小于阈值的框，交并比大于阈值的框被抑制了
        inds = np.where(ovr <= thresh)[0] 
        # 出开了第一个也就是得分最大的那个, 选出的order依然是按照有大到小的顺序排列
        # 继续从剩下的框中重复之前的计算，知道没有可用框
        order = order[inds + 1] 
    
    return keep

# 训练代码示例

```python

from dataloaders import dataloader
from models import cptn_base
from keras.optimizers import Adam 
from keras.callbacks import ModelCheckpoint
import os
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from keras.backend.tensorflow_backend import set_session 
config = tf.ConfigProto() 
config.gpu_options.allow_growth=True
set_session(tf.Session(config=config))


textdir = 'datasets/train_labels'
imgdir  = 'datasets/train_images'

train_gen = dataloader.gen_sample(textdir, imgdir)

model = cptn_base.cptn_model((None, None, 3))
model.compile(
    optimizer=Adam(1e-5),
    loss={'rpn_class_reshape': cptn_base.rpn_loss_clas, 'rpn_regress_reshape': cptn_base.rpn_loss_regr},
    loss_weights={'rpn_class_reshape': 1.0, 'rpn_regress_reshape': 1.0}
)

callbacks = [
    ModelCheckpoint(r'weights/weights-ctpnlstm-{epoch:02d}.hdf5',
                    save_weights_only=True)
]

model.fit_generator(
    train_gen,
    epochs=20,
    steps_per_epoch=6000,
    callbacks=callbacks
)
```

# 预测代码示例

```python
from dataloaders import dataloader
from models import cptn_base
from keras.optimizers import Adam 
from keras.callbacks import ModelCheckpoint
from keras import layers
from keras.models import Model
import os
import tensorflow as tf
import cv2
import numpy as np
from dataloaders import dataloader, text_line_bulder
import pickle
import matplotlib.pyplot as plt
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


IMAGE_MEAN = [123.68,116.779,103.939]


# 给训练网络加类别输出加一个激活函数，以便得到类别的评分
model = cptn_base.cptn_model((None, None, 3))
clas, regr = model.output
input_tensor = model.input
clas_prod = layers.Activation('softmax', name='rpn_cls_softmax')(clas)

predict_model = Model(input_tensor, [clas, regr, clas_prod])
predict_model.load_weights('weights/weights-ctpnlstm-20_20190412_0817.hdf5')

# test_img = cv2.imread('datasets/train_images/100.jpg')
test_img = cv2.imread('/home/y/文档/神经网络数据集/Challenge2_Test_Task12_Images/img_14.jpg')
h, w, c = test_img.shape
pred_img = test_img - IMAGE_MEAN
pred_img = pred_img[np.newaxis, ...]

# 预测
clas, regr, clas_pord = predict_model.predict(pred_img)

# 通过基本anchor和预测的regr还原预测的bbox
anchor = dataloader.gen_anchor((int(h / 16), int(w / 16)), 16)
bbox = dataloader.bbox_transfrom_inv(anchor, regr)
bbox = dataloader.clip_box(bbox, [h, w])

# 选出类别评分大于0.7的框
fg = np.where(clas_pord[0, :, 1] > 0.7)[0]
select_anchor = bbox[fg, :]
select_score = clas_pord[0, fg, 1]
select_anchor = select_anchor.astype('int32')

# 过滤掉宽和高小于阈值的框
keep_index = dataloader.filter_bbox(select_anchor, 16)
select_anchor = select_anchor[keep_index]
select_score  = select_score[keep_index]

# 极大值抑制
select_score = np.reshape(select_score, (select_score.shape[0], 1)) # shape=(N,) -> shape=(N, 1)
nmsbox = np.hstack([select_anchor, select_score]) # shape=(N, 5) [N, [x1, y1, x2, y2, score]]
keep = dataloader.nms(nmsbox, 0.3) # 计算出留下的anchor的下标
select_anchor = select_anchor[keep]
select_score  = select_score[keep]

# 使用文本线构造算法，把检测出的小框合并为大框
text_recs = text_line_bulder.get_text_lines(select_anchor, select_score, (h, w))
text_recs = text_recs.astype('int32')

for i in text_recs:
    cv2.line(test_img, (i[0], i[1]), (i[2], i[3]), (255, 0, 0), 2)
    cv2.line(test_img, (i[0], i[1]), (i[4], i[5]), (255, 0, 0), 2)
    cv2.line(test_img, (i[6], i[7]), (i[2], i[3]), (255, 0, 0), 2)
    cv2.line(test_img, (i[4], i[5]), (i[6], i[7]), (255, 0, 0), 2)

# for i in select_anchor:
#     cv2.rectangle(test_img, (i[0], i[1]), (i[2], i[3]), (0, 255, 0))

plt.imshow(test_img[..., ::-1])
plt.show()
```